In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from os import listdir
from os.path import isfile, join
import tqdm
import tensorflow as tf
from tensorflow import keras
import mysql.connector as msql
from mysql.connector import Error

In [2]:
# the datasets are too large, do not push to github!
# download the file on the desktop
mypath = "data"
files = [mypath+'/'+str(f) for f in listdir(mypath) if isfile(join(mypath, f))]


Create data dictionary and move these 5 files into it.

Output of listdir(mypath) should be:

data/members_v3.csv.7z 

data/sample_submission_zero.csv.7z

data/train.csv.7z

data/transactions.csv.7z

data/user_logs.csv.7z

In [3]:
files

['data/members_v3.csv.7z',
 'data/sample_submission_zero.csv.7z',
 'data/train.csv.7z',
 'data/transactions.csv.7z']

In [4]:
if "unpacked_data" not in listdir('./'):
    os.makedirs("./unpacked_data")
    unpacked = False
else:
    print("Dictionary is already created")
    unpacked = True

Dictionary is already created


In [6]:
# unpack data
import py7zr

if unpacked is False:
    extract_path = "unpacked_data"
    for file in tqdm.tqdm(files):
        with py7zr.SevenZipFile(file, mode='r') as z:
            data = z.extractall(extract_path)

In [4]:
#user_data = pd.read_csv("unpacked_data/user_logs.csv")
member_dat = pd.read_csv("unpacked_data/members_v3.csv")
transactions_dat = pd.read_csv('unpacked_data/transactions.csv')
train_dat = pd.read_csv('unpacked_data/train.csv')

In [5]:
'''
Functions that use to change the datatype according to columns' largest values for memory saving。

For integer:
For example, the maximum number that int8 can store is 127, and the minimum is - 128;
if the largest values for such column is smaller than the value，
we change the data type to int8
Perform this for all columns include int.

For float:
We change all features to float32
Source: https://www.kaggle.com/jeru666/did-you-think-of-these-features/notebook
'''

def change_datatype(df):
    int_cols = list(df.select_dtypes(include=["int64"]).columns)
    for col in int_cols:
        if ((np.max(df[col]) <= 127) and(np.min(df[col] >= -128))):
            df[col] = df[col].astype(np.int8)
        elif ((np.max(df[col]) <= 32767) and(np.min(df[col] >= -32768))):
            df[col] = df[col].astype(np.int16)
        elif ((np.max(df[col]) <= 2147483647) and(np.min(df[col] >= -2147483648))):
            df[col] = df[col].astype(np.int32)
        else:
            df[col] = df[col].astype(np.int64)
            
def change_datatype_float(df):
    float_cols = list(df.select_dtypes(include=['float']).columns)
    for col in float_cols:
        df[col] = df[col].astype(np.float32)

In [6]:
transactions_dat.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,20150930,20151031,0
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,41,30,129,129,1,20150930,20160427,0
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,39,30,149,149,1,20150930,20151128,0
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,39,30,149,149,1,20150930,20151121,0


The memory used before reducation

In [14]:
mem = transactions_dat.memory_usage(index=True).sum()
print("The memory usage before reduction is", mem /1024**2,"MB")

The memory usage before reduction is 513.7384204864502 MB


We change the datatype of transactions_dat for memory reduction.

In [15]:
change_datatype(transactions_dat)

The memory used after reduction

In [16]:
mem = transactions_dat.memory_usage(index=True).sum()
print("The memory usage before reduction is", mem /1024**2,"MB")

The memory usage before reduction is 513.7384204864502 MB


### Feature Engineering with MySQL

Avoid running this section if the engineered_data.csv is already existed, engineering the first million records took 4 hours and the dataset contains more than 20 millions records.

In [18]:
try:
    engineered_df = pd.read_csv("engineered_data.csv")
except:
    pass

In [96]:
import mysql.connector as msql
from mysql.connector import Error

def connect_sql(password, database = None):
    if database == None:
        conn = msql.connect(host='localhost', user='root',  
                        password=password)#give ur username, password
    else:
        conn = msql.connect(host='localhost', user='root',  
                        password=password, database = database)

        print("You're connected to database: ", database)
    return conn


passward = "Chen090718"

try:
    conn = connect_sql(passward)
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE churn_prediction")
        print("Database is created")
except Error as e:
    print("Error while connecting to MySQL", e)

Error while connecting to MySQL 1007 (HY000): Can't create database 'churn_prediction'; database exists


Input transaction data into SQL

In [20]:
transactions_dat.dtypes

msno                      object
payment_method_id           int8
payment_plan_days          int16
plan_list_price            int16
actual_amount_paid         int16
is_auto_renew               int8
transaction_date           int32
membership_expire_date     int32
is_cancel                   int8
dtype: object

In [21]:
train_df = pd.merge(transactions_dat, train_dat, on = 'msno') # merge the datasets

In [22]:
# create the transactions table in sql server for later use

try:
    conn = connect_sql(passward, "churn_prediction")
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute('DROP TABLE IF EXISTS transactions;')
        sql = ('CREATE TABLE transactions(msno varchar(255),'+
                'payment_method_id int,payment_plan_days int,plan_list_price int, actual_amount_paid int,is_auto_renew int,'+
                'transaction_date int, membership_expire_date int, is_cancel int, is_churn int)')
        cursor.execute(sql)
        #loop through the data frame
        for i,row in tqdm.tqdm(train_df.iterrows()):
            #here %S means string values 
            sql = "INSERT INTO churn_prediction.transactions VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            # the connection is not auto committed by default, so we must commit to save our changes
            conn.commit()
except Error as e:
            print("Error while connecting to MySQL", e)

0it [00:00, ?it/s]

You're connected to database:  churn_prediction
Creating table....
Table is created....


15883148it [35:26:15, 124.50it/s]


In [23]:
sql_avg = '''SELECT 
                msno, 
                COUNT(payment_method_id) as n_transc, 
                AVG(payment_plan_days) as mean_plan_days, 
                AVG(plan_list_price) as mean_list_price, 
                AVG(actual_amount_paid) as mean_amount_paid,
                MAX(is_cancel)
            FROM churn_prediction.transactions 
            GROUP BY msno'''

In [103]:
conn = connect_sql(passward, "churn_prediction")
cursor = conn.cursor()
cursor.execute(sql_avg)
# Fetch all the records
result = cursor.fetchall()

You're connected to database:  churn_prediction


In [104]:
avg_df = pd.DataFrame(result, columns=['msno', 'n_transc', 'mean_plan_days','mean_list_price', 'mean_amount_paid', "is_cancel"])

In [105]:
sql_payment_mode = '''SELECT msno, payment_method_id, MAX(occurs)
                      FROM (SELECT msno,payment_method_id, count(*) as occurs
                            FROM churn_prediction.transactions
                            GROUP BY msno, payment_method_id
                      ) T1
                      GROUP BY msno'''

In [106]:
cursor.execute(sql_payment_mode)
# Fetch all the records
result = cursor.fetchall()

In [107]:
payment_df = pd.DataFrame(result, columns=['msno', 'payment_method_id', 'occurs']).drop("occurs", axis = 1)

In [108]:
sql_auto_mode = '''SELECT msno, is_auto_renew, MAX(occurs)
                      FROM (SELECT msno,is_auto_renew, count(*) as occurs
                            FROM churn_prediction.transactions
                            GROUP BY msno, is_auto_renew
                      ) T1
                      GROUP BY msno'''

In [109]:
cursor.execute(sql_auto_mode)
# Fetch all the records
result = cursor.fetchall()

In [110]:
auto_df = pd.DataFrame(result, columns=['msno', 'payment_method_id', 'occurs']).drop("occurs", axis = 1)

In [111]:
sql_date = '''SELECT o.msno, o.transaction_date, MAX(o.payment_plan_days), MAX(o.plan_list_price), MAX(o.actual_amount_paid), o.is_churn
              FROM churn_prediction.transactions o
                  LEFT JOIN churn_prediction.transactions b
                      ON o.msno = b.msno AND o.transaction_date <  b.transaction_date
              WHERE b.transaction_date is NULL
              GROUP BY o.msno
             '''

In [112]:
cursor.execute(sql_date)
# Fetch all the records
result = cursor.fetchall()

In [113]:
date_df = pd.DataFrame(result, columns=['msno', 'last_transaction_date', 'last_payment_days', 'last_list_price', 'last_payment', "is_churn"])

In [114]:
engineered_df = avg_df.merge(payment_df, on="msno").merge(auto_df, on="msno").merge(date_df, on="msno")

In [115]:
engineered_df.to_csv("engineered_data.csv")

### Merge data and explore correlation

In [12]:
train_dat = pd.read_csv('unpacked_data/train.csv')

In [ ]:
train1 = pd.merge(train_dat, user_data, on = 'msno') 
train2 = pd.merge(train_dat, member_dat, on = 'msno') 
train3 = pd.merge(train_dat, transactions_dat, on = 'msno') 
    # system crack if merge all

In [ ]:
train2['gender'].replace({'female':'1', 'male':'0'}, inplace = True)
train1.head()
train2.head()
train3.head()

In [ ]:
corr = train1.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
corr = train2.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
corr = train3.corr()
corr.style.background_gradient(cmap='coolwarm')

## Logistic Regression Model

In [116]:
# Import package
from sklearn.linear_model import LogisticRegression

In [117]:
X = train1.iloc[:,2:]
y = train1.loc[:,['is_churn']]
lr = LogisticRegression(random_state=0).fit(X, y)
lr.predict(X)
lr.score(X, y) # Output: 0.9442647435443151

NameError: name 'train1' is not defined

In [ ]:
X = train2.iloc[:,2:]
y = train2.loc[:,['is_churn']]
lr = LogisticRegression(random_state=0).fit(X, y)
lr.predict(X)
lr.score(X, y) # Need to debug, dataset has Null value???

In [ ]:
X = train3.iloc[:,2:]
y = train3.loc[:,['is_churn']]
lr = LogisticRegression(random_state=0).fit(X, y)
lr.predict(X)
lr.score(X, y) # Output: 0.9568365792473885

Using the engineered data

In [118]:
from sklearn.model_selection import train_test_split

In [119]:
dat = engineered_df.iloc[:,2:].sample(frac = 1)
train, test = train_test_split(dat, test_size=0.2)

In [120]:
train_y = train['is_churn'].values
train_X = train.drop("is_churn", axis = 1).values
test_y = test['is_churn'].values
test_X = test.drop("is_churn", axis = 1).values

In [121]:
lr = LogisticRegression(random_state=0).fit(train_X, train_y)
lr.predict(test_X)
lr.score(test_X, test_y)

0.9359877534783244

### Random Forest

In [122]:
#import packages
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [123]:
clf=RandomForestClassifier(n_estimators=100)
clf.fit(train_X,train_y)
Y_pred=clf.predict(test_X)

In [124]:
print("Accuracy:",metrics.accuracy_score(test_y, Y_pred))

Accuracy: 0.9557070704527487


### Neural Network

In [125]:
model = keras.Sequential([

    keras.layers.Dense(11, activation='relu'),

    keras.layers.Dense(8, activation='relu'),
    
    keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [126]:
train_X = np.asarray(train_X).astype('float32')
train_y = np.asarray(train_y).astype('float32')

In [127]:
model.fit(train_X, train_y, epochs=4, batch_size=16)

Epoch 1/4
49647/49647 [==============================] - 31s 627us/step - loss: 957240634769408.0000 - accuracy: 0.9361
Epoch 2/4
49647/49647 [==============================] - 33s 668us/step - loss: 0.2376 - accuracy: 0.9361
Epoch 3/4
49647/49647 [==============================] - 34s 693us/step - loss: 0.2376 - accuracy: 0.9361
Epoch 4/4
49647/49647 [==============================] - 34s 690us/step - loss: 0.2376 - accuracy: 0.9361


In [128]:
test_X = np.asarray(test_X).astype('float32')
test_y = np.asarray(test_y).astype('float32')
test_loss, test_acc = model.evaluate(test_X, test_y, verbose=1)

6206/6206 [==============================] - 4s 664us/step - loss: 0.2379 - accuracy: 0.9360


In [153]:
predictions = model.predict(test_X)

In [154]:
predictions

array([[0.9559264 , 0.04407364],
       [0.9559264 , 0.04407364],
       [0.9559264 , 0.04407364],
       ...,
       [0.9559264 , 0.04407364],
       [0.9559264 , 0.04407364],
       [0.9559264 , 0.04407364]], dtype=float32)